# Imports

In [1]:
import pickle
from skimage import io

# Code From Zach

In [2]:
input_directory = "./data/processed/"
with open(input_directory + "PhotoTable.p", 'rb') as pickle_file:
    photo_info = pickle.load(pickle_file)
photo_info

,Filename,City,Target
File,,,
1_1,1_1.jpg,Chicago,False
1_2,1_2.jpg,Chicago,False
1_3,1_3.jpg,Chicago,False
1_4,1_4.jpg,Chicago,False
1_5,1_5.jpg,Chicago,False
...,...,...,...
12_6,12_6.jpg,Toronto,False
12_7,12_7.jpg,Toronto,False
12_8,12_8.jpg,Toronto,False


In [3]:
sample_photo = io.imread(input_directory + photo_info.loc['1_1', 'Filename'])
io.imshow(sample_photo)
sample_photo.shape

FileNotFoundError: No such file: '/Users/carlos/Documents/GitHub/NYC-Detective/data/processed/1_1.jpg'

# model setup

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense  , Activation
from keras import applications
import re
import os
from PIL import Image


In [ ]:
img_width = 150
img_height = 150
train = input_directory

In [4]:
training = [train+i for i in os.listdir(train)]
training[:5]

NameError: name 'os' is not defined

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]
training.sort(key=natural_keys)
training[:5]

In [ ]:
#there are new york 
training[50:60]

In [ ]:
def prepare_data(list_of_images):
    x = [] # images as arrays
    for image in list_of_images:
        x.append(img_to_array(load_img(image,target_size=sample_photo.shape)))
    return x

In [ ]:
X=prepare_data(training[:-1])
X=np.array(X)
y=np.array([0]*3000 + [1]*3000)

In [ ]:
X.shape

In [ ]:
batch_size = 16
model = Sequential()
#input layer
model.add(Flatten(input_shape=X.shape[1:]))

# first hidden layer
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))

# Second hidden layer
model.add(Dense(units=126,activation='relu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

#fitting the model
history=model.fit(x=X ,y= photo_info.Target,
                  epochs=10,
                  batch_size=batch_size,
                  )

In [ ]:
model.save('city_v1.h5')


In [ ]:
import keras
city_model = keras.models.load_model('./city_v1.h5')

In [ ]:
input_directory = "./data/test/"
testing = [input_directory+i for i in os.listdir(input_directory)]
testing.sort(key=natural_keys)
Xt=prepare_data(testing[:])
Xt=np.array(Xt)

In [ ]:
Xt.shape

In [ ]:
pred = city_model.predict(Xt)

In [ ]:
pred